## Müllabfuhr


[Aufgabe](muellabfuhr.pdf) - [Lösungshinweise](muellabfuhrL.pdf)

### Kürzeste Wege mit Floyd

Der Floyd-Algorithmus liefert uns kürzeste Wege zwischen zwei beliebigen Knoten.
Die beiden beiden berechneten Matrizen D (Distanzmatrix) und P (Vorgängerknoten des kürzesten Wegs)
speichern wir als globale Variablen. Der Floyd-Algorithmus erwartet als Eingabe eine Adjazenzmatrix


#### Adjazenzmatrix erstellen

In [1]:
inf = float('inf')
nr = 6
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]

G = [[inf]*anzV for _ in range(anzV)]
V = set()
E = set()
for i in range(anzE):
    von, bis, km = [int(x) for x in f.readline().split()]
    G[von][bis]=km
    G[bis][von]=km
    V.add(von)
    V.add(bis)
    
    if von < bis:
        E.add((von,bis))
    else:
        E.add((bis,von))
for i in range(anzV):
    G[i][i]=0

In [11]:
G

[[0, inf, inf, inf, 6, 6, 1, inf],
 [inf, 0, inf, 9, inf, inf, 1, inf],
 [inf, inf, 0, 7, inf, inf, inf, inf],
 [inf, 9, 7, 0, 7, 3, 1, inf],
 [6, inf, inf, 7, 0, 5, inf, 8],
 [6, inf, inf, 3, 5, 0, inf, 2],
 [1, 1, inf, 1, inf, inf, 0, 1],
 [inf, inf, inf, inf, 8, 2, 1, 0]]

In [2]:
def floyd(c):
    n = len(c)
    d = [[0]*n for j in range(n)]   # Matrix D
    p = [[0]*n for j in range(n)]   # Matrix P
    for i in range(n):
        for j in range(n):
            d[i][j] = c[i][j]
            p[i][j] = i

    for k in range(n):
        for i in range(n):
            for j in range(n):
                tmp = d[i][k] + d[k][j]
                if tmp < d[i][j]:
                    d[i][j] = tmp
                    p[i][j] = p[k][j]
    return d, p

def getPath(p, i, j):
    if i == j: return (i,)
    return getPath(p,i,p[i][j]) + (j,)

def printMatrix(a):
    for i in range(len(a)):
        for j in range(len(a)):
            if a[i][j] == inf:
                print("{:4}".format("."),end='')
            else:
                print("{:<4}".format(a[i][j]),end='')
        print()
    print()
    
D, P = floyd(G)

In [ ]:
printMatrix(D)

In [17]:
printMatrix(P

0   6   3   6   0   7   0   6   
6   1   3   6   0   7   1   6   
6   6   2   2   3   3   3   6   
6   6   3   3   3   3   3   6   
4   6   3   4   4   4   0   5   
6   6   3   5   5   5   7   5   
6   6   3   6   0   7   6   6   
6   6   3   6   5   7   7   7   



In [24]:
print(sPath(0,2))
print(dist(0,2))

(0, 6, 3, 2)
9


### Kreis, der alle Kanten durchläuft.

Ein Kreis, der alle Kanten nur einmal durchläuft, ist ein Eulerkreis. Der existiert genau dann, wenn wir alle Knoten einen geraden Grad haben. Das können wir bei unseren Graphen nicht erwarten. Wir verbinden die Knoten, die ungeraden Grad haben paarweise miteinander (möglichst billige Strecken), dann konstruieren wir den Eulerkreis. Dazu benötigen wir eine Modellierung des Graphen als Adjazenzliste. Allerdings brauchen wir hier die Gewichte nicht berücksichtigen (wir wollen ja alle Kanten durchlaufen, egal was sie für ein Gewicht haben). Nur beim matching der ungeraden Kanten benötigen wir die Information über die kürzesten Wege. Die holen wir uns n  

In [4]:
# die Matrizen D und P werden als globale Variablen vorausgesetzt.
def dist(v1, v2):
    '''
    returns: kürzeste Distanz zwischen v1 und v2  
    
    '''
    return D[v1][v2]

def sPath(v1, v2):
    '''
    returns: einen kürzesten Weg zwischen v1 und v2
    '''
    return getPath(P,v1,v2)

In [5]:
def match(U):
    '''
    returns: greedy match
    '''
    if len(U) == 0:
        return []
    v = U[0]
    best = 1
    best_val = dist(U[0],U[1])
    for i in range(2,len(U)):
        if U[i] not in G[U[0]]:
            val = dist(U[0],U[i])
            if val < best_val:
                best_val = val
                best = i
    e = (U[0],U[best])
    U1 = U.copy()
    U1.pop(best)
    U1.pop(0)
    return match(U1) + [e]

def abschnitt(p,i,j):
    '''
    returns: Kosten für den Weg 0 - p[i]-p[j] - 0
    '''
    pcost = 0
    for k in range(i,j):

        pcost += G[p[k]][p[k+1]]
    return dist(0,p[i]) + dist(p[j],0) + pcost

def aufteilung(path,n):
    tmp = []
    i = 0
    j = 0
    for k in range(5):
        while j < len(path) and (abschnitt(path,i,j)) <= n:
            j+=1
        tmp.append((i,j-1))
        i=j-1
        j=i+1
    return tmp

#### Einlesen des Graphen mit Adjazenzsets

Wir haben unter Umständen Mehrfachkanten, deswegen 

In [15]:
nr = 6
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]
G = {x:dict() for x in range(anzV)}
for _ in range(anzE):
    v1, v2, cost = f.readline().split()
    v1, v2, cost = int(v1), int(v2), int(cost)
    G[v1][v2] = cost
    G[v2][v1] = cost
 

#Knoten mit ungeradem Grad bestimmen und ein gieriges Matching durchführen

U = []
for v in G:
    if len(G[v]) % 2 != 0:
        U.append(v)
len(U)
print(U)

meta = match(U)
meta, len(meta)

[0, 1, 2, 3, 4, 5, 7, 8, 12, 13, 14, 15, 21, 22, 24, 28, 29, 30, 33, 34, 35, 37, 39, 40, 42, 43, 46, 48, 49, 51, 52, 55, 57, 60, 62, 63, 64, 68, 69, 70, 72, 73, 75, 78, 81, 83, 86, 87, 88, 90, 91, 95, 98, 99]


([(90, 99),
  (83, 88),
  (69, 78),
  (64, 81),
  (57, 62),
  (52, 55),
  (51, 63),
  (46, 98),
  (40, 43),
  (39, 48),
  (37, 75),
  (34, 73),
  (30, 70),
  (29, 95),
  (28, 33),
  (22, 24),
  (15, 49),
  (13, 86),
  (12, 87),
  (8, 14),
  (7, 21),
  (5, 72),
  (4, 35),
  (3, 91),
  (2, 42),
  (1, 68),
  (0, 60)],
 27)

In [14]:
def matchCost(U):
    cost = 0
    for v,w in match(U):
        print(dist(v,w))
        cost += dist(v,w)
    return cost

print(f'Kosten des Matchings {matchCost(U)}')

62164
61682
13450
16358
39715
11038
19552
11600
6734
17186
28638
23051
17540
10479
22554
15071
26486
26059
17559
21455
17450
25097
11827
10917
24586
7256
11583
Kosten des Matchings 577087


Metakanten einführen. Dazu Metaknoten einführen, weil unsere Modellierung keine doppelten Kanten kann


    

In [7]:
hilf = 10000
for e in meta:
    v1, v2 = e
    G[hilf] = dict()
    G[v1][hilf] = G[v2][hilf] = dist(v1,v2)
    G[hilf][v2] = G[hilf][v1] = 0
    hilf+=1
    
 
degrees = {x:len(G[x]) for x in G} 

In [8]:
sum(degrees.values())

516

In [9]:
# Kontrolle, ob alle Knoten geraden Grad haben
for v in G:
    if len(G[v]) % 2 != 0:
        print(v)

In [10]:
import sys
from copy import deepcopy
GE = deepcopy(G)
degrees = {x:len(GE[x]) for x in GE} 
def dfs(v):

    while degrees[v] != 0:
        next = list(GE[v])[0]  # die erste noch vorhandene Kante
        del(GE[v][next])     # wird aus dem Graphen gelöscht
        del(GE[next][v])     # von beiden Seiten
        degrees[v]-=1         # der Grad der Knoten v und next
        degrees[next]-=1      # wird erniedrigt
        dfs(next)
    path.append(v) 
zaehl = 0
path = []
dfs(0)
path.reverse()                # beim undirected eulercircle unnötig
print(path)    

[0, 4, 44, 0, 58, 45, 41, 5, 2, 13, 5, 67, 41, 42, 5, 77, 13, 84, 2, 72, 13, 10017, 86, 42, 45, 10, 4, 93, 35, 32, 46, 35, 88, 73, 20, 8, 87, 27, 43, 66, 40, 89, 16, 80, 53, 29, 26, 6, 62, 54, 9, 17, 22, 9, 47, 22, 54, 17, 56, 9, 26, 56, 47, 26, 57, 29, 10013, 95, 53, 57, 92, 74, 16, 92, 79, 57, 10004, 62, 59, 24, 22, 10015, 24, 82, 31, 28, 30, 25, 39, 30, 33, 60, 39, 19, 2, 10024, 42, 67, 77, 18, 23, 69, 38, 23, 78, 55, 36, 34, 8, 10019, 14, 3, 1, 11, 3, 49, 1, 10025, 68, 37, 78, 69, 75, 55, 52, 20, 34, 10011, 73, 52, 10005, 55, 97, 23, 75, 38, 97, 69, 10002, 78, 75, 10010, 37, 11, 36, 14, 68, 49, 21, 12, 7, 15, 12, 50, 15, 85, 12, 10018, 87, 85, 50, 7, 10020, 21, 50, 91, 68, 3, 10023, 91, 49, 10016, 15, 94, 27, 99, 40, 10008, 43, 99, 89, 96, 99, 10000, 90, 32, 81, 51, 65, 81, 76, 63, 64, 51, 10006, 63, 80, 65, 76, 80, 64, 10003, 81, 88, 32, 98, 10, 48, 19, 84, 77, 86, 72, 10021, 5, 86, 41, 90, 88, 10001, 83, 30, 10012, 70, 28, 83, 25, 48, 10009, 39, 83, 33, 10014, 28, 82, 70, 31, 24,

In [13]:
len(path)

269

In [11]:
def eliminateMeta(path):
    for i in range(len(path)-1,-1,-1):
        if path[i] >= 10000:
            zwischen = list(sPath(path[i-1],path[i+1]))
            path = (path[:i]+zwischen[1:-1]+path[i+1:])
    return path
            
path = eliminateMeta(path)
print(path)

[0, 4, 44, 0, 58, 45, 41, 5, 2, 13, 5, 67, 41, 42, 5, 77, 13, 84, 2, 72, 13, 72, 86, 42, 45, 10, 4, 93, 35, 32, 46, 35, 88, 73, 20, 8, 87, 27, 43, 66, 40, 89, 16, 80, 53, 29, 26, 6, 62, 54, 9, 17, 22, 9, 47, 22, 54, 17, 56, 9, 26, 56, 47, 26, 57, 29, 53, 95, 53, 57, 92, 74, 16, 92, 79, 57, 26, 6, 62, 59, 24, 22, 24, 82, 31, 28, 30, 25, 39, 30, 33, 60, 39, 19, 2, 5, 42, 67, 77, 18, 23, 69, 38, 23, 78, 55, 36, 34, 8, 34, 36, 14, 3, 1, 11, 3, 49, 1, 3, 68, 37, 78, 69, 75, 55, 52, 20, 34, 20, 73, 52, 55, 97, 23, 75, 38, 97, 69, 78, 75, 78, 37, 11, 36, 14, 68, 49, 21, 12, 7, 15, 12, 50, 15, 85, 12, 85, 87, 85, 50, 7, 12, 21, 50, 91, 68, 3, 68, 91, 49, 21, 12, 15, 94, 27, 99, 40, 99, 43, 99, 89, 96, 99, 27, 87, 8, 20, 73, 88, 90, 32, 81, 51, 65, 81, 76, 63, 64, 51, 64, 63, 80, 65, 76, 80, 64, 51, 81, 88, 32, 98, 10, 48, 19, 84, 77, 86, 72, 86, 5, 86, 41, 90, 88, 35, 93, 4, 44, 60, 33, 83, 30, 28, 70, 28, 83, 25, 48, 25, 39, 83, 33, 30, 28, 82, 70, 31, 24, 61, 60, 44, 59, 60, 44, 0, 71, 46, 3

In [12]:
len(path)

269

In [8]:
def aufteilungOK(path,n):
    au = aufteilung(path,n)
    lastIndex = au[-1][1]
    return lastIndex == len(path)-1
    

In [9]:
mincost = 0
maxcost = abschnitt(path,0,len(path)-1)
best = maxcost
versuch = maxcost//2
 
while True:
    
    if aufteilungOK(path,versuch):
        best = versuch
        versuch = versuch//2
    else:
        versuch1 = (versuch+best) // 2
        if versuch1 == versuch: 
             break
        versuch = versuch1
    
 
   
print(best)
aufteilung(path,best),len(path)

21


([(0, 2), (2, 5), (5, 7), (7, 11), (11, 18)], 19)